##########################################################################
# KNN
##########################################################################

### 1. Exploratory Data Analysis
### 2. Data Preprocessing
### 3. Model & Prediction
### 4. Model Evaluation
### 5. Hyperparameter Optimization
### 6. Final Model

In [27]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate

In [28]:
pd.set_option("display.max_columns", None)

##################################################################
# 1.Exploratory Data Analysis - VERİ ANALİZİ
##################################################################

In [29]:
df = pd.read_csv("DATA/diabetes.csv")

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [7]:
df["Outcome"].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

###################################################################################
# 2. Data Preprocessing - VERİ ÖN İŞLEME
###################################################################################

In [9]:
y = df["Outcome"]

In [10]:
X = df.drop(["Outcome"], axis=1)

### şimdi uzaklık olduğu için bu yöntemde standartlaşma yapmamız gerekiyor çünkü daha doğru aonuçlar alabilmek için

In [11]:
X_scaled = StandardScaler().fit_transform(X)

In [13]:
X = pd.DataFrame(X_scaled, columns=X.columns)

In [14]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


##############################################################################################
# 3. Modeling & Prediction - MODELLEME
###############################################################################################

In [15]:
knn_model = KNeighborsClassifier().fit(X, y)

In [16]:
random_user = X.sample(1, random_state=45)

In [18]:
random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
195,0.342981,1.161295,0.770014,1.283638,1.130518,0.940144,-0.232176,-0.360847


In [19]:
knn_model.predict(random_user)

array([1], dtype=int64)

#########################################################################################################
# 4. Model Evaluation - MODEL BAŞARI DEĞERLENDİRME
#########################################################################################################

### karmaşıklık matrisi için bütün modelin model tahmini yapılır yani y_pred bulunur.

In [20]:
y_pred = knn_model.predict(X)

### roc eğrisinin altında kalan alanı hesaplamak için auc değeri hesaplanır bunun için de bize olasılık değerleri (1,0) olma durumları gerekir.

In [21]:
y_prob = knn_model.predict_proba(X)[:, 1]

### y_pred ile classification_report değerleri yani accuracy recall f1-score precision değerleri bulunur.

In [23]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



### precision = 1 sınıfına ait başarı oranı
### recall= gerçekte 1 olan değerlerin 1 olma oranı
### f1-score0 ikisinin harmanlanmış halli
### accuracy = başarılı sınıflandırma oranı yani (1-1 durumu ve 0-0 durumu)

In [24]:
roc_auc_score(y, y_prob)

0.9017686567164179

### CAPRAZ doğrulama işlemi yapacağız bunlar için sırayla model, bağımsız,bağımlı değişken, cv= kaç katlı çd , kullanacağın yöntem

In [30]:
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

In [31]:
cv_results

{'fit_time': array([0.02517819, 0.00352025, 0.004287  , 0.00349927, 0.00329471]),
 'score_time': array([0.05508137, 0.03114915, 0.02775788, 0.02893305, 0.0309999 ]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [32]:
cv_results["test_accuracy"].mean()

0.733112638994992

In [33]:
cv_results["test_f1"].mean()

0.5905780011534191

In [34]:
cv_results["test_roc_auc"].mean()

0.7805279524807827

### cross validate modeli daha doğrular yani daha doğru sonuçlar verir.

## Bu elde ettiğimiz başarı sonuçları nasıl arttırılabilir ? 
### 1) gözlem sayısı arttırılabilir.
### 2) veri ön işleme daha detaylandırılabilir.
### 3) yeni değişkenler türetilebilir - özellik mühendisliği
### 4) ilgili algoritma için optimizasyonlar yapılabilir.


###############################################################################################################
# 5. Hyperparameter Optimization - HİPERPARAMETRE OPTİMİZASYONU
###############################################################################################################

### Dışarıdan kullanıcının ayarlaması gereken parametreleri programatik şekilde en doğru nasıl ayarlanır 

In [35]:
knn_model = KNeighborsClassifier()

In [36]:
knn_model

KNeighborsClassifier()

In [38]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

### mesela küme sayısının = n_neighbors': 5 olduğunu veriyor peki bunu nasıl değiştirebiliriz.

In [39]:
knn_params = {"n_neighbors": range(2, 50)}

In [41]:
knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


### knn_gs_best = optimum komsu sayısı knn metodunda 

In [42]:
knn_gs_best.best_params_

{'n_neighbors': 17}

####################################################################################################
# 6. Final Model
####################################################################################################

In [43]:
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

In [44]:
cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [45]:
cv_results

{'fit_time': array([0.02558374, 0.01981711, 0.0039947 , 0.00848866, 0.00252938]),
 'score_time': array([0.15922213, 0.05920792, 0.03851175, 0.04637003, 0.04277229]),
 'test_accuracy': array([0.75324675, 0.74025974, 0.75974026, 0.81045752, 0.77124183]),
 'test_f1': array([0.60416667, 0.60784314, 0.57471264, 0.70103093, 0.59770115]),
 'test_roc_auc': array([0.80675926, 0.7912037 , 0.79166667, 0.84603774, 0.82830189])}

In [46]:
cv_results["test_accuracy"].mean()

0.7669892199303965

In [47]:
cv_results["test_f1"].mean()

0.6170909049720137

In [48]:
cv_results["test_roc_auc"].mean()

0.8127938504542278

In [ ]:
örneğin


In [51]:
random_user = X.sample(1)

In [52]:
knn_final.predict(random_user)

array([0], dtype=int64)